Modelo de Machine Learning - Sistema de Recomendacion de Peliculas

Vamos a construir un sistema de recomendación basado en similitud de ítems utilizando CountVectorizer para convertir las características de las películas en vectores numéricos y la similitud del coseno para calcular la similitud entre estas películas. Este enfoque es bastante común para sistemas de recomendación basado en Contenido

In [3]:
# Importamos las Librerias a Usar

import pandas as pd
import pyarrow as pq
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
# Cragar los Datos

movies = pd.read_parquet('DataSets\df_movies_limpio.parquet')
credits = pd.read_parquet('DataSets\df_credits_limpio.parquet')
genres = pd.read_parquet('DataSets\df_genres_limpio.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'DataSets\\df_movies_limpio.parquet'

In [ ]:


# 'id' es la columna común entre los DataFrames
movies = movies.merge(credits, on='id')
movies = movies.merge(genres, on='id')

# Crear una nueva columna combinando el género y los actores
movies['combination'] = movies['genres'].apply(lambda x: ' '.join(eval(x))) + ' ' + movies['cast'].apply(lambda x: ' '.join(eval(x)[:3]))  # Usar los 3 primeros actores

# 3. Vectorizar las características
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(movies['combination'])

# 4. Calcular la similitud del coseno
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 5. Crear una función para obtener recomendaciones
def get_recommendations(title, cosine_sim=cosine_sim):
    # Obtener el índice de la película que coincide con el título
    idx = movies[movies['title'] == title].index[0]  # Cambia 'title' por el nombre de tu columna

    # Obtener las puntuaciones de similitud de esa película con todas las demás
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas basadas en la puntuación de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las 10 películas más similares
    sim_scores = sim_scores[1:11]  # Excluye la primera (la misma película)
    movie_indices = [i[0] for i in sim_scores]

    # Retornar las 10 películas más similares
    return movies['title'].iloc[movie_indices]

# Funcion 

titulo_pelicula = 'Toy Story'   
recomendaciones = get_recommendations(titulo_pelicula)
print(f'Recomendaciones para {titulo_pelicula}:')
print(recomendaciones)